In [ ]:
%cd /content
!pip install torchsde einops diffusers transformers accelerate peft timm kornia aiohttp
!apt install -qqy

!git clone https://github.com/comfyanonymous/ComfyUI /content/ComfyUI
!git clone https://github.com/ltdrdata/ComfyUI-Manager /content/ComfyUI/custom_nodes/ComfyUI-Manager
!git clone https://github.com/lrzjason/Comfyui-In-Context-Lora-Utils /content/ComfyUI/custom_nodes/Comfyui-In-Context-Lora-Utils
!git clone https://github.com/cubiq/ComfyUI_essentials /content/ComfyUI/custom_nodes/ComfyUI_essentials

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev.sft -d /content/ComfyUI/models/unet -o flux1-dev.sft
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/ComfyUI/models/clip -o clip_l.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp16.safetensors -d /content/ComfyUI/models/clip -o t5xxl_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/ComfyUI/models/vae -o ae.sft

In [ ]:
%cd /content/ComfyUI

import os, shutil, json, requests, random, time
from urllib.parse import urlsplit

import torch
from PIL import Image
import numpy as np

from nodes import load_custom_node
from nodes import NODE_CLASS_MAPPINGS
from comfy_extras import nodes_model_advanced, nodes_custom_sampler, nodes_flux

load_custom_node("/content/ComfyUI/custom_nodes/Comfyui-In-Context-Lora-Utils")
load_custom_node("/content/ComfyUI/custom_nodes/ComfyUI_essentials")

UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
LoraLoader = NODE_CLASS_MAPPINGS["LoraLoader"]()
LoadImage = NODE_CLASS_MAPPINGS["LoadImage"]()
AddMaskForICLora = NODE_CLASS_MAPPINGS["AddMaskForICLora"]()
ModelSamplingFlux = nodes_model_advanced.NODE_CLASS_MAPPINGS["ModelSamplingFlux"]()
CLIPTextEncodeFlux = nodes_flux.NODE_CLASS_MAPPINGS["CLIPTextEncodeFlux"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SetLatentNoiseMask = NODE_CLASS_MAPPINGS["SetLatentNoiseMask"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAEEncode  = NODE_CLASS_MAPPINGS["VAEEncode"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
ImageCrop = NODE_CLASS_MAPPINGS["ImageCrop+"]()

with torch.inference_mode():
    unet = UNETLoader.load_unet("flux1-dev-fp8.safetensors", "fp8_e4m3fn")[0]
    clip = DualCLIPLoader.load_clip("t5xxl_fp16.safetensors", "clip_l.safetensors", "flux")[0]
    vae = VAELoader.load_vae("ae.sft")[0]

def download_file(url, save_dir, file_name):
    os.makedirs(save_dir, exist_ok=True)
    file_suffix = os.path.splitext(urlsplit(url).path)[1]
    file_name_with_suffix = file_name + file_suffix
    file_path = os.path.join(save_dir, file_name_with_suffix)
    response = requests.get(url)
    response.raise_for_status()
    with open(file_path, 'wb') as file:
        file.write(response.content)
    return file_path

@torch.inference_mode()
def generate(input):
    values = input["input"]

    input_image = values['input_image_check']
    input_image = download_file(url=input_image, save_dir='/content/ComfyUI/input', file_name='input_image')
    positive_prompt = values['positive_prompt']
    seed = values['seed']
    steps = values['steps']
    guidance = values['guidance']
    sampler_name = values['sampler_name']
    scheduler = values['scheduler']
    max_shift = values['max_shift']
    base_shift = values['base_shift']
    patch_mode = values['patch_mode']
    output_length = values['output_length']
    patch_color = values['patch_color']
    custom_lora1_url = values['custom_lora1_url']
    custom_lora1_file = download_file(url=custom_lora1_url, save_dir='/content/ComfyUI/models/loras', file_name='custom_lora1_file')
    custom_lora1_file = os.path.basename(custom_lora1_file)
    custom_lora1_strength_model = values['custom_lora1_strength_model']
    custom_lora1_strength_clip = values['custom_lora1_strength_clip']
    custom_lora2_url = values['custom_lora2_url']
    custom_lora2_file = download_file(url=custom_lora2_url, save_dir='/content/ComfyUI/models/loras', file_name='custom_lora2_file')
    custom_lora2_file = os.path.basename(custom_lora2_file)
    custom_lora2_strength_model = values['custom_lora2_strength_model']
    custom_lora2_strength_clip = values['custom_lora2_strength_clip']

    if seed == 0:
        random.seed(int(time.time()))
        seed = random.randint(0, 18446744073709551615)
    print(seed)

    custom_lora1_unet, custom_lora1_clip = LoraLoader.load_lora(unet, clip, custom_lora1_file, custom_lora1_strength_model, custom_lora1_strength_clip)
    lora_unet, lora_clip = LoraLoader.load_lora(custom_lora1_unet, custom_lora1_clip, custom_lora2_file, custom_lora2_strength_model, custom_lora2_strength_clip)
    conditioning = CLIPTextEncodeFlux.encode(lora_clip, positive_prompt, positive_prompt, guidance)[0]
    input_image = LoadImage.load_image(input_image)[0]
    return_images, return_masks, min_x, min_y, target_width, target_height, width, height = AddMaskForICLora.add_mask(input_image, patch_mode, output_length, patch_color)
    final_model = ModelSamplingFlux.patch(lora_unet, max_shift, base_shift, width, height)[0]
    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(final_model, conditioning)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(lora_unet, scheduler, steps, 1.0)[0]
    latent_image = VAEEncode.encode(vae, return_images)[0]
    latent_image = SetLatentNoiseMask.set_mask(latent_image, return_masks)[0]
    sample, _ = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    decoded = VAEDecode.decode(vae, sample)[0].detach()
    decoded = ImageCrop.execute(decoded, target_width, target_height, "top-left", min_x, min_y)[0]
    image = Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save(f"/content/in-context-lora-{seed}-tost.png")

    result = f"/content/in-context-lora-{seed}-tost.png"

    return result

In [ ]:
input = { 
    "input": {
        "input_image_check": "https://files.catbox.moe/z6mdko.png",
        "positive_prompt": "This pair of images shows a pattern as [IMAGE1] Em1ru holding a packaging bag [IMAGE2]; the pattern on the packaging bag in [IMAGE2] must refer to [IMAGE1]; [IMAGE1] Em1ru holding a packaging bag; the packaging bag in [IMAGE2] is printed with [IMAGE1]; [IMAGE2] Em1ru holds a packaging bag with the [IMAGE1] pattern printed on it.",
        "seed": 0,
        "steps": 40,
        "guidance": 3.5,
        "sampler_name": "euler",
        "scheduler": "simple",
        "max_shift": 1.15,
        "base_shift": 0.50,
        "patch_mode": "auto",
        "output_length": 1536,
        "patch_color": "#FF0000",
        "custom_lora1_url": "https://files.catbox.moe/4zgu8u.safetensors",
        "custom_lora1_strength_model": 1.0,
        "custom_lora1_strength_clip": 1.0,
        "custom_lora2_url": "https://files.catbox.moe/e2hsa6.safetensors",
        "custom_lora2_strength_model": 0.9,
        "custom_lora2_strength_clip": 0.9
    }
}
image = generate(input)
Image.open(image)